In [2]:
import os 
os.chdir('c:\\Users\\abbaig\\Krish naik course\\heart-disease-prediction')

In [3]:
import pandas as pd
df= pd.read_csv('artifacts/data_ingestion/bank_marketing_updated_v1.csv')

C:\Users\abbaig\AppData\Local\Temp\ipykernel_21576\3085845448.py:2: DtypeWarning: Columns (0,1,2,3,11,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df= pd.read_csv('artifacts/data_ingestion/bank_marketing_updated_v1.csv')


In [4]:
df.head()

,banking marketing,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,customer id and age.,NaN,Customer salary and balance.,NaN,Customer marital status and job with education...,NaN,particular customer before targeted or not,NaN,Loan types: loans or housing loans,NaN,Contact type,NaN,month of contact,duration of call,NaN,NaN,NaN,outcome of previous contact,response of customer after call happned
1,customerid,age,salary,balance,marital,jobedu,targeted,default,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,response
2,1,58,100000,2143,married,"management,tertiary",yes,no,yes,no,unknown,5,"may, 2017",261 sec,1,-1,0,unknown,no
3,2,44,60000,29,single,"technician,secondary",yes,no,yes,no,unknown,5,"may, 2017",151 sec,1,-1,0,unknown,no
4,3,33,120000,2,married,"entrepreneur,secondary",yes,no,yes,yes,unknown,5,"may, 2017",76 sec,1,-1,0,unknown,no


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270 entries, 0 to 269
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   age            270 non-null    int64
 1   sex            270 non-null    int64
 2   BP             270 non-null    int64
 3   cholestrol     270 non-null    int64
 4   heart disease  270 non-null    int64
dtypes: int64(5)
memory usage: 10.7 KB


dtype('int64')

In [5]:
df.columns

Index(['banking marketing', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3',
       'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8',
       'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12',
       'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16',
       'Unnamed: 17', 'Unnamed: 18'],
      dtype='object')

In [6]:
## update the entity
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataValidationConfig:
    root_dir: Path
    unzip_data_dir: Path
    STATUS_FILE: Path
    all_schema: dict   


In [7]:
from heart_disease_project.utils.common import read_yaml,create_directories
from heart_disease_project.constants import *

In [8]:

##update the configuration manager
class ConfigurationManager:
    def __init__(self,config_file_path=CONFIG_FILE_PATH,params_file_path=PARAMS_FILE_PATH,schema_file_path=SCHEMA_FILE_PATH):
        self.config=read_yaml(CONFIG_FILE_PATH)
        self.params=read_yaml(PARAMS_FILE_PATH)
        self.schema=read_yaml(SCHEMA_FILE_PATH)

        create_directories([self.config.data_validation.root_dir])

    def get_data_validation_config(self)-> DataValidationConfig:
        config=self.config.data_validation
        schema= self.schema.COLUMNS
        
        data_validation_config=DataValidationConfig(
            root_dir= config.root_dir,
            unzip_data_dir= config.unzip_data_dir,
            STATUS_FILE= config.STATUS_FILE,
            all_schema= schema
            
        )

        return data_validation_config

In [34]:
##update the component
import os 
from heart_disease_project import logger 

class DataValidation:
    def __init__(self,config: DataValidationConfig):
        self.config= config

    def validate_all_columns(self)-> bool:
        try: 
            validation_status:None
            data = pd.read_csv(self.config.unzip_data_dir)
            data = data.iloc[1:]
            data.columns=data.iloc[0]
            all_cols= list(data.columns)
            all_schema = self.config.all_schema.keys()
            
            for col in all_cols:
                if col not in all_schema:
                    validation_status=False
                    with open(self.config.STATUS_FILE,'w') as f:
                        f.write(f"validation status: {validation_status}")
                if col in all_schema:
                    validation_status=True
                    with open(self.config.STATUS_FILE,'w')as f:
                        f.write(f"validation_status: {validation_status}")
                
            return validation_status,data.columns
        
        except Exception as e:
            raise e

In [35]:
## update the pipeline
config= ConfigurationManager()
data_validation_config= config.get_data_validation_config()
data_validation = DataValidation(config=data_validation_config)
data_validation.validate_all_columns()

[2024-03-25 13:03:56,316: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-25 13:03:56,321: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-25 13:03:56,329: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-03-25 13:03:56,332: INFO: common: created directory at: artifacts/data_validation]


C:\Users\abbaig\AppData\Local\Temp\ipykernel_21576\3677495727.py:12: DtypeWarning: Columns (0,1,2,3,11,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(self.config.unzip_data_dir)


(True,
 Index(['customerid', 'age', 'salary', 'balance', 'marital', 'jobedu',
        'targeted', 'default', 'housing', 'loan', 'contact', 'day', 'month',
        'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'response'],
       dtype='object', name=1))